# XProj demo

In [1]:
import numpy as np
import shapely
import xarray as xr
import xvec

import xproj

xr.set_options(display_expand_indexes=True);

## Example datasets

- Vector data cube (https://xvec.readthedocs.io) with no CRS attached

In [2]:
random_points = shapely.points(np.random.uniform(-180, 180, 50), np.random.uniform(-90, 90, 50))

ds_vec = xr.Dataset(coords={"points": ("space", random_points)}).set_xindex(
    "points", xvec.GeometryIndex
)

ds_vec

<xarray.Dataset> Size: 400B
Dimensions:  (space: 50)
Coordinates:
  * points   (space) object 400B POINT (140.62012177466158 81.50307804679687)...
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points   GeometryIndex (crs=None)

## Set the CRS

`.proj.set_crs()` can be used to set (or update) the CRS: it creates a new scalar coordinate (if it doesn't exist yet) with a `xproj.CRSIndex`.

Note: the name of the CRS coordinate is abritrary.

In [3]:
ds_wgs84 = ds_vec.proj.set_crs(spatial_ref="epsg:4326")

ds_wgs84

<xarray.Dataset> Size: 408B
Dimensions:      (space: 50)
Coordinates:
  * points       (space) object 400B POINT (140.62012177466158 81.50307804679...
  * spatial_ref  int64 8B 0
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points       GeometryIndex (crs=None)
    spatial_ref  CRSIndex (crs=EPSG:4326)

## "Link" the CRS coordinate with other, indexed geospatial coordinates

TODO.

## Get the CRS

Via the `.proj.crs` property, which returns a [pyproj.CRS](https://pyproj4.github.io/pyproj/stable/api/crs/crs.html) object.

In [4]:
ds_wgs84.proj.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Alternatively, it is possible to explicitly pass the CRS coordinate name to the `.proj` accessor:

In [5]:
ds_wgs84.proj("spatial_ref").crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## CRS-aware alignment

The index of the CRS coordinate is used to compare or align xarray objects based on their CRS.

In [6]:
ds_pseudo_mercator = ds_wgs84.proj.set_crs(spatial_ref="epsg:3857", allow_override=True)

In [7]:
ds_wgs84 + ds_pseudo_mercator

MergeError: conflicting values/indexes on objects to be combined for coordinate 'spatial_ref'
first index: CRSIndex
<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

second index: CRSIndex
<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

first variable: <xarray.Variable ()> Size: 8B
array(0)
second variable: <xarray.Variable ()> Size: 8B
array(0

## Unset the CRS

Just drop the CRS index and/or coordinate

In [8]:
ds_no_crs = ds_wgs84.drop_vars("spatial_ref")

ds_no_crs

<xarray.Dataset> Size: 400B
Dimensions:  (space: 50)
Coordinates:
  * points   (space) object 400B POINT (140.62012177466158 81.50307804679687)...
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points   GeometryIndex (crs=None)

Note: possible to align or merge datasets with and without a defined CRS

In [9]:
ds_wgs84 + ds_no_crs

<xarray.Dataset> Size: 408B
Dimensions:      (space: 50)
Coordinates:
  * points       (space) object 400B POINT (140.62012177466158 81.50307804679...
  * spatial_ref  int64 8B 0
Dimensions without coordinates: space
Data variables:
    *empty*
Indexes:
    points       GeometryIndex (crs=None)
    spatial_ref  CRSIndex (crs=EPSG:4326)

## Only one unique CRS per Dataset or DataArray

Xproj currently supports only one CRS defined per Dataset or DataArray.

In [10]:
# TODO: actually not well enforced yet

ds_multiple_crss = ds_wgs84.proj.set_crs(other_spatial_ref="epsg:3857")

ds_multiple_crss.proj("other_spatial_ref").crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich